In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
X = pd.read_csv('face_emotion_data_X.csv', header=None).values
y = pd.read_csv('face_emotion_data_Y.csv', header=None).values

rand = 1100

### Truncated SVD 

Truncated SVD solution. Use the pseudo-inverse V Σ−1UT , where Σ−1 is com- puted by inverting the k largest singular values and setting others to zero. Here, k is the regularization parameter and it takes values k = 1, 2, . . . , 9; i.e., compute 9 different solutions, wk.

In [3]:
def svd_reg_lstsq(A, b, k):
    [u, s, vt] = np.linalg.svd(A)
    rank = np.linalg.matrix_rank(A)
    
    sigma = np.zeros((rank, rank))
    np.fill_diagonal(sigma, s)
    
    gamma  = np.zeros((rank, rank))
    np.fill_diagonal(gamma, s[:k])
    
    middle = sigma @ np.linalg.inv(sigma**2 + gamma)

    return u @ middle @ vt[:rank, :] @ b


In [4]:
def truncated_regularized_lstsq(X, y):
    # hold weights for each k
    wi = []
    error_rate = []
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 30, test_size=.25, random_state=rand)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size= 30, test_size=0.5, random_state=rand)
    
    # training
    for k in range(1, 9):
        w_temp = svd_reg_lstsq(X_train.T, y_train, k)
    
        wi.append(w_temp)
        
        
    # validation
    for i in range(0,8):
        new_predictions =  [0 if i < .5 else 1 if i < 1.5 else 2 for i in (X_val @ wi[i])]
        error_rate.append(1-sum([1 if i == True else 0 for i in (np.array(y_val).ravel() == new_predictions)]) /
                          len(y_val))
    
    # test result
    best_k = error_rate.index(min(error_rate))
    new_predictions =  [0 if i < .5 else 1 if i < 1.5 else 2 for i in X_test.dot(wi[best_k])]
    
    
    return 1-sum([1 if i == True else 0 for i in (np.array(y_val).ravel() == new_predictions)]) / len(y_test)

truncated_regularized_lstsq(X, y)

0.875

### Regularised SVD

Regularized LS. Let w􏰀λ = arg minw ∥y − Xw∥2 + λ∥w∥2, for the following values of the regularization parameter λ = 0,2−1,20,21,22,23, and 24. Show that w􏰀λ can be computed using the SVD and use this fact in your code.

In [5]:
def reg_lstsq(A, b, gamma):
    num_rows, num_cols = A.shape
    
     # (A^T*A + gamma*I)^-1 * A^T * b
    g  = np.zeros((num_cols, num_cols))
    np.fill_diagonal(g, gamma)
    return np.linalg.inv(A.transpose().dot(A) + g).dot(A.transpose()).dot(b)

In [6]:
def regularized_svd(X, y):
    # hold weights for each k
    gamma = [0, 1/2, 1, 4, 8, 16]
    
    wi = []
    error_rate = []
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = .75, test_size=.25)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, test_size=0.5)
    
    # training
    for g in gamma:
        w_temp = reg_lstsq(X_train, y_train, g)
    
        wi.append(w_temp)
    
    # validation
    for w in wi:
        new_predictions =  [0 if i < .5 else 1 if i < 1.5 else 2 for i in X_val.dot(w)]
        error_rate.append(1-sum([1 if i == True else 0 for i in (np.array(y_val).ravel() == new_predictions)]) /
                          len(y_val))
    
    # test result
    best_w = error_rate.index(min(error_rate))
    new_predictions =  [0 if i < .5 else 1 if i < 1.5 else 2 for i in X_test.dot(wi[best_w])]
    
    
    return 1-sum([1 if i == True else 0 for i in (np.array(y_val).ravel() == new_predictions)]) / len(y_test)
regularized_svd(X, y)

0.9375